In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error

In [5]:
#import the data
df = pd.read_excel("../../../site_info_ver_3.2.xlsx")
df.reset_index(drop=True,inplace=True)

In [9]:
# Some functions to help out with
def plot_predictions(test,predicted):
    plt.plot(test, color='red',label='Real Entries')
    plt.plot(predicted, color='blue',label='Predicted Entries')
    plt.title('Entries Prediction')
    plt.xlabel('Time')
    plt.ylabel('Total Entries')
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

Prepering the DF

In [10]:
#only 1 site
site_df = df.loc[df.Site_Name == 'Alexander Stream-Bet Yannai']
pollution_cols = ['pm10', 'pm2.5', 'nox', 'so2',
        'Tel_Aviv-Yafo_pm10', 'Jerusalem_pm10',
       'Haifa_pm10', 'Ashkelon_pm10', 'Beer-Sheva_pm10', 'Tel_Aviv-Yafo_pm2.5',
       'Jerusalem_pm2.5', 'Haifa_pm2.5', 'Ashkelon_pm2.5', 'Beer-Sheva_pm2.5',
       'Tel_Aviv-Yafo_nox', 'Jerusalem_nox', 'Haifa_nox', 'Ashkelon_nox',
       'Beer-Sheva_nox', 'Tel_Aviv-Yafo_so2', 'Jerusalem_so2', 'Haifa_so2',
       'Ashkelon_so2', 'Beer-Sheva_so2', 'is_Site_exceeded_pm10',
       'is_Site_exceeded_pm2.5', 'is_Site_exceeded_nox',
       'is_Site_exceeded_so2', 'Tel_Aviv-Yafo_pm10_exceeded',
       'Jerusalem_pm10_exceeded', 'Haifa_pm10_exceeded',
       'Ashkelon_pm10_exceeded', 'Beer-Sheva_pm10_exceeded',
       'Tel_Aviv-Yafo_pm2.5_exceeded', 'Jerusalem_pm2.5_exceeded',
       'Haifa_pm2.5_exceeded', 'Ashkelon_pm2.5_exceeded',
       'Beer-Sheva_pm2.5_exceeded', 'Tel_Aviv-Yafo_so2_exceeded',
       'Jerusalem_so2_exceeded', 'Haifa_so2_exceeded', 'Ashkelon_so2_exceeded',
       'Beer-Sheva_so2_exceeded', 'Tel_Aviv-Yafo_nox_exceeded',
       'Jerusalem_nox_exceeded', 'Haifa_nox_exceeded', 'Ashkelon_nox_exceeded',
       'Beer-Sheva_nox_exceeded','Israelis_Count','Tourists_Count']
site_df.drop(pollution_cols ,axis=1, inplace=True)

C:\Users\asars\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [11]:
# site_df.drop(['Site_Name', 'Israelis_Count' , 'Tourists_Count'] ,axis=1, inplace=True)
site_df.set_index('Date' , drop=True , inplace=True)

In [36]:
# Checking for missing value
y = site_df.Total
X = site_df.drop('Total',axis=1)
scaler_train = MinMaxScaler()
scaler_test = MinMaxScaler()
scaler_traget_train = MinMaxScaler()
scaler_traget_test = MinMaxScaler()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=False,random_state=False)

train_df= pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)



,Site_Name,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,operations,is_jewish_holiday,is_jewish_vacation,is_muslims_holiday,...,is_water,is_cave,is_lookout,Temperature,is_HeatWave,Green_border,Season_autumn,Season_spring,Season_summer,Season_winter
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,Alexander Stream-Bet Yannai,1,0,0,0,1,0,0,0,0,...,1,0,0,15.5,0,0,0,0,0,1
2016-01-02,Alexander Stream-Bet Yannai,1,0,0,0,1,0,0,0,0,...,1,0,0,13.8,0,0,0,0,0,1
2016-01-03,Alexander Stream-Bet Yannai,1,0,0,0,0,0,0,0,0,...,1,0,0,16.6,0,0,0,0,0,1
2016-01-04,Alexander Stream-Bet Yannai,1,0,0,0,0,0,0,0,0,...,1,0,0,17.5,0,0,0,0,0,1
2016-01-05,Alexander Stream-Bet Yannai,1,0,0,0,0,0,0,0,0,...,1,0,0,18.9,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-07,Alexander Stream-Bet Yannai,1,0,0,0,1,0,0,1,0,...,1,0,0,42.4,1,0,0,0,1,0
2021-08-08,Alexander Stream-Bet Yannai,1,0,0,0,0,0,0,1,0,...,1,0,0,37.9,1,0,0,0,1,0
2021-08-09,Alexander Stream-Bet Yannai,1,0,0,0,0,0,0,1,1,...,1,0,0,38.0,1,0,0,0,1,0


In [ ]:
# The GRU architecture
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Fourth GRU layer
regressorGRU.add(GRU(units=50, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=1))
# Compiling the RNN
regressorGRU.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
# Fitting to the training set
regressorGRU.fit(X_train,y_train,epochs=50,batch_size=150)

C:\Users\liavc\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/50
6/6 [==============================] - 7s 99ms/step - loss: 8265059.0000
Epoch 2/50
6/6 [==============================] - 1s 93ms/step - loss: 8248178.5000
Epoch 3/50
6/6 [==============================] - 1s 94ms/step - loss: 8097861.5000
Epoch 4/50
6/6 [==============================] - 1s 94ms/step - loss: 8169680.0000
Epoch 5/50
6/6 [==============================] - 1s 92ms/step - loss: 8211965.0000
Epoch 6/50
6/6 [==============================] - 1s 93ms/step - loss: 8222077.5000
Epoch 7/50
6/6 [==============================] - 1s 93ms/step - loss: 8139602.0000
Epoch 8/50
6/6 [==============================] - 1s 93ms/step - loss: 8225671.0000
Epoch 9/50
6/6 [==============================] - 1s 98ms/step - loss: 8257678.5000
Epoch 10/50
6/6 [==============================] - 1s 101ms/step - loss: 8444314.0000
Epoch 11/50
6/6 [==============================] - 1s 112ms/step - loss: 8530089.0000
Epoch 12/50
6/6 [==============================] - 1s 94ms/step - loss: 

In [ ]:
X_test = []
for i in range(60,311):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
GRU_predicted_stock_price = regressorGRU.predict(X_test)
GRU_predicted_stock_price = sc.inverse_transform(GRU_predicted_stock_price)